The traffic accident data is the key to this analysis. To make sure it's current, instead of downloading a static .csv file it, along with several other datasets from the City of Nashville's OpenData portal, will be periodically downloaded and updated via APIs.

To avoid too many pull requests, all of the data will be pulled here and the relevant information will be exported for use in a separate notebook for EDA.

** This notebook will be revisited and edited as data needs may change for this analysis.

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
# Needed to pip install sodapy first
from sodapy import Socrata 

**Data Source:** https://data.nashville.gov/Police/Traffic-Accidents/6v6w-hpcw/about_data

In [2]:
client = Socrata("data.nashville.gov", None)

In [3]:
crashes_export = client.get("6v6w-hpcw", limit=200000)
crashes_raw = pd.DataFrame.from_records(crashes_export)
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,reporting_officer,collision_type,collision_type_description,weather,...,rpa,precinct,lat,long,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,property_damage
0,20240113671,2024-02-18T18:00:00.000,2,0,0,False,4008412,5,SIDESWIPE - SAME DIRECTION,21,...,8927,SOUTH,36.0488,-86.613,"{'type': 'Point', 'coordinates': [-86.613, 36....",1,1,33,15,NaN
1,20240113561,2024-02-18T17:38:00.000,2,0,0,False,4007344,4,ANGLE,21,...,5325,NORTH,36.1691,-86.8096,"{'type': 'Point', 'coordinates': [-86.8096, 36...",1,1,25,51,NaN
2,20240113525,2024-02-18T17:23:00.000,1,2,0,False,4008013,0,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,21,...,9561,HERMIT,36.1798,-86.6134,"{'type': 'Point', 'coordinates': [-86.6134, 36...",1,2,7,44,NaN
3,20240113457,2024-02-18T16:50:00.000,2,0,0,False,4007988,4,ANGLE,21,...,4105,CENTRA,36.1615,-86.776,"{'type': 'Point', 'coordinates': [-86.776, 36....",1,1,20,29,NaN
4,20240113169,2024-02-18T12:49:00.000,2,0,0,False,4004452,11,Front to Rear,21,...,1801,EAST,36.2347,-86.7248,"{'type': 'Point', 'coordinates': [-86.7248, 36...",1,1,15,23,NaN


In [4]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180299 entries, 0 to 180298
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   accident_number              180299 non-null  object
 1   date_and_time                180299 non-null  object
 2   number_of_motor_vehicles     180298 non-null  object
 3   number_of_injuries           180299 non-null  object
 4   number_of_fatalities         180299 non-null  object
 5   hit_and_run                  180285 non-null  object
 6   reporting_officer            180288 non-null  object
 7   collision_type               180289 non-null  object
 8   collision_type_description   180289 non-null  object
 9   weather                      174248 non-null  object
 10  weather_description          174248 non-null  object
 11  illuaccidemination           180019 non-null  object
 12  illumination_description     180019 non-null  object
 13  harmfulcodes  

Now that the DataFrame has been created, some columns can be removed as they're not relevant to this analysis.

In [5]:
crashes_raw = crashes_raw.drop(['reporting_officer', 'collision_type', 'illuaccidemination', 'harmfulcodes', ':@computed_region_wvby_4s8j', ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq', ':@computed_region_gxvr_9jxz', 'weather'], axis=1)

In [6]:
crashes_raw['date_and_time'] = pd.to_datetime(crashes_raw['date_and_time'])
crashes_raw = crashes_raw.astype({'number_of_motor_vehicles': 'float',
                    'number_of_injuries': 'float',
                    'number_of_fatalities': 'float',
                    'hit_and_run': 'bool',
                    'property_damage': 'bool'})

In [7]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180299 entries, 0 to 180298
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   accident_number             180299 non-null  object        
 1   date_and_time               180299 non-null  datetime64[ns]
 2   number_of_motor_vehicles    180298 non-null  float64       
 3   number_of_injuries          180299 non-null  float64       
 4   number_of_fatalities        180299 non-null  float64       
 5   hit_and_run                 180299 non-null  bool          
 6   collision_type_description  180289 non-null  object        
 7   weather_description         174248 non-null  object        
 8   illumination_description    180019 non-null  object        
 9   harmfuldescriptions         178228 non-null  object        
 10  street_address              180294 non-null  object        
 11  city                        180299 non-

For some initial cleaning, all text fields should be converted to the same case, in this case UPPER

In [8]:
crashes_raw['collision_type_description'] = crashes_raw['collision_type_description'].astype(str).str.upper()
crashes_raw['illumination_description'] = crashes_raw['illumination_description'].astype(str).str.upper()
crashes_raw['harmfuldescriptions'] = crashes_raw['harmfuldescriptions'].astype(str).str.upper()
crashes_raw['street_address'] = crashes_raw['street_address'].astype(str).str.upper()
crashes_raw['city'] = crashes_raw['city'].astype(str).str.upper()
crashes_raw['state'] = crashes_raw['state'].astype(str).str.upper()
crashes_raw['precinct'] = crashes_raw['precinct'].astype(str).str.upper()
crashes_raw['weather_description'] = crashes_raw['weather_description'].astype(str).str.upper()
crashes_raw['property_damage'] = crashes_raw['property_damage'].astype(str).str.upper()

In [9]:
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,collision_type_description,weather_description,illumination_description,harmfuldescriptions,street_address,city,state,zip,rpa,precinct,lat,long,mapped_location,property_damage
0,20240113671,2024-02-18 18:00:00,2.0,0.0,0.0,False,SIDESWIPE - SAME DIRECTION,CLEAR,DARK-UNKNOWN LIGHTING,MOTOR VEHICLE IN TRANSPORT,US HWY 41 & MOUNTAIN SPRINGS RD,ANTIOCH,TN,37013,8927,SOUTH,36.0488,-86.613,"{'type': 'Point', 'coordinates': [-86.613, 36....",TRUE
1,20240113561,2024-02-18 17:38:00,2.0,0.0,0.0,False,ANGLE,CLEAR,DUSK,MOTOR VEHICLE IN TRANSPORT,21ST AVN & JEFFERSON ST,NASHVILLE,TN,37208,5325,NORTH,36.1691,-86.8096,"{'type': 'Point', 'coordinates': [-86.8096, 36...",TRUE
2,20240113525,2024-02-18 17:23:00,1.0,2.0,0.0,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,CLEAR,DAYLIGHT,DITCH;RAN OFF ROAD-RIGHT,DODSON CHAPEL RD & MAGNUM DR,HERMITAGE,TN,37076,9561,HERMIT,36.1798,-86.6134,"{'type': 'Point', 'coordinates': [-86.6134, 36...",TRUE
3,20240113457,2024-02-18 16:50:00,2.0,0.0,0.0,False,ANGLE,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,3RD AVS & BROADWAY,NASHVILLE,TN,37201,4105,CENTRA,36.1615,-86.776,"{'type': 'Point', 'coordinates': [-86.776, 36....",TRUE
4,20240113169,2024-02-18 12:49:00,2.0,0.0,0.0,False,FRONT TO REAR,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,BRILEY PKWY E EXT RAMP & GALLATIN PKE,NASHVILLE,TN,37216,1801,EAST,36.2347,-86.7248,"{'type': 'Point', 'coordinates': [-86.7248, 36...",TRUE


Last, this analysis is only concerned with crashes in the vicinity of Main Street / Gallatin Avenue / Gallatin Pike in East Nashville & Madison. To filter the data, a list of points needs to be created to essentially trace the path of the corridor. From there, only data points within, say, a 150' radius of that line will be included for analysis. I can't find a better way to do this than selecting points on Google Maps and manually entering the coordinates into a list. Refer to `mapping.ipynb`

In [10]:
east_nash_crashes = crashes_raw[crashes_raw['zip'].isin(['37206', '37216', '37115'])]
east_nash_crashes

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,collision_type_description,weather_description,illumination_description,harmfuldescriptions,street_address,city,state,zip,rpa,precinct,lat,long,mapped_location,property_damage
4,20240113169,2024-02-18 12:49:00,2.0,0.0,0.0,False,FRONT TO REAR,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,BRILEY PKWY E EXT RAMP & GALLATIN PKE,NASHVILLE,TN,37216,1801,EAST,36.2347,-86.7248,"{'type': 'Point', 'coordinates': [-86.7248, 36...",TRUE
15,20240112411,2024-02-18 00:51:00,2.0,0.0,0.0,False,ANGLE,CLEAR,DARK - LIGHTED,MOTOR VEHICLE IN TRANSPORT,STATE RT 45 & GALLATIN PKEN,MADISON,TN,37115,1701,MADISO,36.2631,-86.7119,"{'type': 'Point', 'coordinates': [-86.7119, 36...",TRUE
19,20240112142,2024-02-17 20:49:00,1.0,0.0,0.0,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,CLEAR,DARK - LIGHTED,NAN,GALLATIN PKEN & OLD TIME VILLAGE PVTDR,MADISON,TN,37115,1701,MADISO,36.2638,-86.7117,"{'type': 'Point', 'coordinates': [-86.7117, 36...",TRUE
31,20240111599,2024-02-17 18:11:00,2.0,0.0,0.0,True,FRONT TO REAR,CLEAR,DARK - LIGHTED,MOTOR VEHICLE IN TRANSPORT,DICKERSON PKE & DUE WEST AVN,MADISON,TN,37115,2017,MADISO,36.2563,-86.7579,"{'type': 'Point', 'coordinates': [-86.7579, 36...",TRUE
63,20240110010,2024-02-16 21:36:00,2.0,0.0,0.0,False,ANGLE,RAIN,DARK - NOT LIGHTED,MOTOR VEHICLE IN TRANSPORT,N 8TH ST & RAMSEY ST,NASHVILLE,TN,37206,1999,EAST,36.1764,-86.7594,"{'type': 'Point', 'coordinates': [-86.7594, 36...",TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180210,20170001270,2017-01-01 14:59:00,2.0,0.0,0.0,False,FRONT TO REAR,RAIN,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,CLEVELAND ST & N 9TH ST,NASHVILLE,TN,37206,1925,EAST,36.1847,-86.7583,"{'type': 'Point', 'coordinates': [-86.7583, 36...",TRUE
180219,20170001226,2017-01-01 14:33:00,2.0,2.0,0.0,False,FRONT TO REAR,RAIN,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,ANDERSON LN & MYATT DR,MADISON,TN,37115,1713,MADISO,36.2721,-86.6890,"{'type': 'Point', 'coordinates': [-86.689, 36....",TRUE
180244,20170000705,2017-01-01 07:59:00,1.0,2.0,0.0,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,CLEAR,DAYLIGHT,GUARDRAIL FACE,I65 S EXT RAMP & I 65,MADISON,TN,37115,20044,MADISO,36.2481,-86.7430,"{'type': 'Point', 'coordinates': [-86.743, 36....",TRUE
180258,20170000450,2017-01-01 03:47:00,1.0,0.0,0.0,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,CLEAR,DARK - LIGHTED,WALL,RIVERWOOD DR & COOPER LN,NASHVILLE,TN,37216,1449,EAST,36.2095,-86.7135,"{'type': 'Point', 'coordinates': [-86.7135, 36...",TRUE


The resulting table is exported to a .csv file for use in the EDA notebook.

In [11]:
east_nash_crashes.to_csv('../data/clean/east_nash_crashes.csv')

Now repeat the process for active ROW permits.<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Active-Right-of-Way-Permits/a5tp-4w2v/about_data

In [12]:
row_permits_export = client.get("a5tp-4w2v", limit=2000000)
row_permits_raw = pd.DataFrame.from_records(row_permits_export)
row_permits_raw.head()

,initiated_date,permit,permit_type,permit_description,status,on_street,from_street,to_street,location_address,city,...,company,days_to_work,latitude,longitude,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,:@computed_region_b9k3_hpc2
0,2023-02-09T11:47:34.000,202305185,DRIVEWAY PERMIT,POURING ENTRANCE AND APRON TO DRIVEWAY,ACTIVE,4108 WESTLAWN DR,WESTLAWN PL,WESTLAWN CT,4108 WESTLAWN DR,NASHVILLE,...,Christy Conway,Monday Tuesday Wednesday Thursday Friday Satur...,36.1393235314126,-86.836318688412,"{'type': 'Point', 'coordinates': [-86.83631868...",1,1,10,46,5
1,2023-06-02T11:01:59.000,202320188,DRIVEWAY PERMIT,RESIDENTIAL DRIVEWAY CONSTRUCTION,ACTIVE,0 KNIGHT DR,MID-BLOCK,JUDY CREEK RD,0 KNIGHT DR,WHITES CREEK,...,Michael L Shular,Monday Tuesday Wednesday Thursday Friday Satur...,36.2626428157731,-86.8253323554335,"{'type': 'Point', 'coordinates': [-86.82533235...",1,2,16,6,1
2,2022-10-06T09:36:29.000,202236666,EXCAVATION PERMIT,STORMWATER INSTALL,ACTIVE,KINGS LN,PHIPPS DR,DRAKES BRANCH RD,KINGS LN,NASHVILLE,...,Middle Tennessee Infrastructure,Tuesday Wednesday Thursday Friday Saturday,36.2175270092533,-86.846531521226,"{'type': 'Point', 'coordinates': [-86.84653152...",1,1,1,34,1
3,2023-04-13T09:09:13.000,202313772,EXCAVATION PERMIT,SEWER/WATER TAP,ACTIVE,4317 CATO RD,CATO RD,GILMORE CROSSING LN,4317 CATO RD,NASHVILLE,...,"R & A PLUMBING, LLC",Monday Tuesday Wednesday Thursday Friday Satur...,36.2138889523239,-86.8666383650662,"{'type': 'Point', 'coordinates': [-86.86663836...",1,1,1,34,1
4,2023-05-15T10:27:04.000,202317822,EXCAVATION PERMIT,PLACING ANCHOR\nJOB# A02G791 (NN)\nSTEP: 1.7,ACTIVE,META DR,WESTCREST DR,ELLENWOOD DR,META DR,NASHVILLE,...,STAR CONSTRUCTION LLC,Tuesday Wednesday Thursday Friday Saturday,36.0594833908917,-86.7349955972504,"{'type': 'Point', 'coordinates': [-86.73499559...",1,1,21,3,3


In [13]:
row_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   initiated_date               25 non-null     object
 1   permit                       25 non-null     object
 2   permit_type                  25 non-null     object
 3   permit_description           25 non-null     object
 4   status                       25 non-null     object
 5   on_street                    25 non-null     object
 6   from_street                  25 non-null     object
 7   to_street                    25 non-null     object
 8   location_address             25 non-null     object
 9   city                         25 non-null     object
 10  zip                          25 non-null     object
 11  scope                        25 non-null     object
 12  scheduled_start              25 non-null     object
 13  scheduled_end                25 non-n

In [14]:
row_permits_raw = row_permits_raw.drop(['status', ':@computed_region_wvby_4s8j', ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq', ':@computed_region_gxvr_9jxz', ':@computed_region_b9k3_hpc2'], axis=1)

In [15]:
row_permits_raw['initiated_date'] = pd.to_datetime(row_permits_raw['initiated_date'])
row_permits_raw['scheduled_start'] = pd.to_datetime(row_permits_raw['scheduled_start'])
row_permits_raw['scheduled_end'] = pd.to_datetime(row_permits_raw['scheduled_end'])

In [16]:
row_permits = row_permits_raw[row_permits_raw['zip'].isin(['37206', '37216', '37115'])]
row_permits

,initiated_date,permit,permit_type,permit_description,on_street,from_street,to_street,location_address,city,zip,...,scheduled_start,scheduled_end,permit_applicant_name,council_district,conditions_traffic_control,company,days_to_work,latitude,longitude,mapped_location
6,2023-07-17 08:29:54,202325417,EXCAVATION PERMIT,SEWER TAP IN ALLEY FOR 1132-B CAHAL AVE,1017 ALY,FOOTPATH,N 16TH ST,1017 ALY,NASHVILLE,37206,...,2023-11-29,2024-02-27,TAMMY GREEN,7,Lane closed maintain two-way traffic Daylight...,PRECISION PLUMBING CO,Monday Tuesday Wednesday Thursday Friday Satur...,36.1956076228412,-86.7389212643167,"{'type': 'Point', 'coordinates': [-86.73892126..."
22,2023-07-17 08:26:53,202325416,EXCAVATION PERMIT,SEWER TAP IN ALLEY FOR 1132-A CAHAL AVE,1017 ALY,FOOTPATH,N 16TH ST,1017 ALY,NASHVILLE,37206,...,2023-11-29,2024-02-27,TAMMY GREEN,7,Lane closed maintain two-way traffic Daylight...,PRECISION PLUMBING CO,Monday Tuesday Wednesday Thursday Friday Satur...,36.1956076228412,-86.7389212643167,"{'type': 'Point', 'coordinates': [-86.73892126..."


In [17]:
row_permits.to_csv('../data/clean/row_permits.csv')

...and Nashville 311 calls<br><br>
Source: https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh/about_data

In [18]:
nash_311_export = client.get("7qhx-rexh", limit=2000000)
nash_311_raw = pd.DataFrame.from_records(nash_311_export)
nash_311_raw.head()

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,...,:@computed_region_v67z_xm3t,:@computed_region_kh5x_g7w5,:@computed_region_cfa7_hbpz,:@computed_region_sjpq_96s8,:@computed_region_gisn_y5cm,:@computed_region_b9k3_hpc2,oem_id,contact_type,parent_case,preferred_language
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-05-04T17:54:30.000,2022-05-04T17:54:30.000,Phone,False,True,...,2,3,71,1527,4,2,NaN,NaN,NaN,NaN
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,2022-04-01T21:01:39.000,2022-04-05T11:29:54.000,Phone,False,False,...,1,9,1018,15550,4,1,NaN,NaN,NaN,NaN
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,2022-04-01T21:04:50.000,2022-04-06T20:14:15.000,Phone,False,False,...,8,33,258,24062,7,8,NaN,NaN,NaN,NaN
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-04-01T21:01:47.000,2022-04-01T21:01:47.000,Phone,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,2022-04-01T21:03:10.000,2022-04-01T22:04:31.000,Phone,False,False,...,7,13,50,30406,1,7,NaN,NaN,NaN,NaN


In [19]:
nash_311_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1434291 entries, 0 to 1434290
Data columns (total 34 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   case_number                  1434291 non-null  object
 1   status                       1434291 non-null  object
 2   case_request                 1434276 non-null  object
 3   case_subrequest              1432955 non-null  object
 4   additional_subrequest        1345459 non-null  object
 5   date_time_opened             1434291 non-null  object
 6   date_time_closed             1419305 non-null  object
 7   case_origin                  1434291 non-null  object
 8   state_issue                  1434291 non-null  object
 9   closed_when_created          1434291 non-null  object
 10  incident_address             975974 non-null   object
 11  incident_city                959510 non-null   object
 12  incident_council_district    961011 non-null   object
 1

In [20]:
nash_311_raw = nash_311_raw.drop(['oem_id', ':@computed_region_wvby_4s8j', ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq', ':@computed_region_gxvr_9jxz', ':@computed_region_yf9r_ed6g', ':@computed_region_fvtq_wnma', ':@computed_region_s8bq_67w7', ':@computed_region_v67z_xm3t', ':@computed_region_kh5x_g7w5', ':@computed_region_cfa7_hbpz', ':@computed_region_sjpq_96s8', ':@computed_region_gisn_y5cm', ':@computed_region_b9k3_hpc2'], axis=1)

In [21]:
nash_311_raw['date_time_opened'] = pd.to_datetime(nash_311_raw['date_time_opened'])
nash_311_raw['date_time_closed'] = pd.to_datetime(nash_311_raw['date_time_closed'])

In [22]:
nash_311 = nash_311_raw[nash_311_raw['incident_zip_code'].isin(['37206', '37216', '37115'])]
nash_311

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location,contact_type,parent_case,preferred_language
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-05-04 17:54:30,2022-05-04 17:54:30,Phone,False,True,1301 Pilgrim Dr,MADISON,8,37115,36.2718801,-86.7175492,"{'latitude': '36.2718801', 'longitude': '-86.7...",NaN,NaN,NaN
30,701976,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,2021-06-07 21:06:00,2021-06-08 20:21:00,Phone,FALSE,FALSE,2218 Greenwood Ave,NASHVILLE,6,37206,36.1877621,-86.7249698,"{'latitude': '36.1877621', 'longitude': '-86.7...",NaN,NaN,NaN
38,695645,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,2021-05-29 20:25:00,2021-06-12 07:19:00,hubNashville Community,FALSE,FALSE,2631 Pennington Ave,NASHVILLE,7,37216,36.19787026,-86.73038612,"{'latitude': '36.19787026', 'longitude': '-86....",NaN,NaN,NaN
39,700804,Closed,Public Safety,Improperly Parked Vehicles,Improperly Parked Vehicles,2021-06-05 22:24:00,2021-06-05 22:43:00,hubNashville Community,FALSE,FALSE,1100 Petway Ave,NASHVILLE,6,37206,36.186285,-86.746726,"{'latitude': '36.186285', 'longitude': '-86.74...",NaN,NaN,NaN
44,609882,Closed,"Trash, Recycling & Litter",Extra Cart Pickup Request,Extra Trash Cart Pickup,2021-03-04 14:38:34,2021-03-04 17:34:22,Phone,False,False,912 Highland Cir,MADISON,3,37115,36.277094,-86.72071179999999,"{'latitude': '36.277094', 'longitude': '-86.72...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434229,1461639,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,NaN,2024-01-22 14:11:52,2024-01-22 15:24:54,Phone,False,False,RAMSEY ST / N 9TH ST,NASHVILLE,5,37206,36.17720728,-86.75730126,"{'latitude': '36.17720728', 'longitude': '-86....",NaN,NaN,NaN
1434232,1461656,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,NaN,2024-01-22 14:12:27,2024-01-22 15:24:16,Phone,False,False,COOPER LN / MOSS ROSE DR,NASHVILLE,7,37216,36.2059619,-86.70032156,"{'latitude': '36.2059619', 'longitude': '-86.7...",NaN,NaN,NaN
1434258,1467990,Closed,"Streets, Roads & Sidewalks",Potholes,Roadway,2024-01-27 15:52:55,2024-02-06 22:20:19,hubNashville Community,False,False,426 Neelys Bend Rd,MADISON,7,37115,36.2551544,-86.7062059,"{'latitude': '36.2551544', 'longitude': '-86.7...",NaN,NaN,NaN
1434276,1469977,Closed,"Streets, Roads & Sidewalks",Potholes,Roadway,2024-01-29 19:47:46,2024-02-09 22:17:40,Phone,False,False,407 S 10th St,NASHVILLE,6,37206,36.1722016,-86.7518623,"{'latitude': '36.1722016', 'longitude': '-86.7...",NaN,NaN,NaN


In [23]:
nash_311.to_csv('../data/clean/nash_311.csv')

...and building permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Building-Permits-Issued/3h5w-q8b7/about_data

In [24]:
bdlg_permits_export = client.get("3h5w-q8b7", limit=2000000)
bdlg_permits_raw = pd.DataFrame.from_records(bdlg_permits_export)
bdlg_permits_raw.head()

,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,state,...,:@computed_region_gisn_y5cm,:@computed_region_v3ji_vzam,:@computed_region_c9xn_skx3,:@computed_region_sjpq_96s8,:@computed_region_kh5x_g7w5,:@computed_region_yf9r_ed6g,:@computed_region_fvtq_wnma,:@computed_region_p6sk_2acq,:@computed_region_b9k3_hpc2,:@computed_region_gxvr_9jxz
0,2019040337,Building Commercial - New,"Kennel / Stable, Kennel / Pound",08000007300,2019-07-09T00:00:00.000,2021-02-24T00:00:00.000,110889,1016 CAMILLA CALDWELL LN,NASHVILLE,TN,...,6,2005,1,16168,1,1,23,1,1,34
1,2020008381,Building Use & Occupancy,"Multifamily, Townhome",05100018100,2020-02-06T00:00:00.000,2021-11-24T00:00:00.000,1,609 CREATIVE WAY,MADISON,TN,...,4,2005,3,1527,3,3,26,3,2,12
2,2020017307,Building Commercial - Rehab,Restaurant (Full Service),10700005000,2020-03-17T00:00:00.000,2021-05-14T00:00:00.000,1449522,1 TERMINAL DR,NASHVILLE,TN,...,3,2005,18,15807,18,18,44,6,9,38
3,2020025536,Building Commercial - New,"College / University, Educational > 12Th",08000002000,2020-04-22T00:00:00.000,2021-08-10T00:00:00.000,1100000,1200 COUNTY HOSPITAL RD,NASHVILLE,TN,...,6,2005,1,16168,1,1,23,1,1,34
4,2020027726,Building Commercial - Rehab,Restaurant (Fast Food),09700011100,2020-05-04T00:00:00.000,2021-04-12T00:00:00.000,500000,5800 OLD HICKORY BLVD,HERMITAGE,TN,...,3,2005,16,28805,16,16,6,26,4,44


In [25]:
bdlg_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33638 entries, 0 to 33637
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   permit                       33638 non-null  object
 1   permit_type_description      33638 non-null  object
 2   permit_subtype_description   33638 non-null  object
 3   parcel                       33638 non-null  object
 4   date_entered                 33638 non-null  object
 5   date_issued                  33638 non-null  object
 6   const_cost                   33637 non-null  object
 7   address                      33638 non-null  object
 8   city                         33636 non-null  object
 9   state                        33638 non-null  object
 10  zip                          33636 non-null  object
 11  subdivision_lot              33637 non-null  object
 12  contact                      33638 non-null  object
 13  per_ty                       33

In [26]:
bdlg_permits_raw = bdlg_permits_raw.drop([':@computed_region_f73m_vb2k', ':@computed_region_cfa7_hbpz', ':@computed_region_gisn_y5cm', ':@computed_region_v3ji_vzam', ':@computed_region_c9xn_skx3', ':@computed_region_sjpq_96s8', ':@computed_region_kh5x_g7w5', ':@computed_region_yf9r_ed6g', ':@computed_region_fvtq_wnma', ':@computed_region_p6sk_2acq', ':@computed_region_b9k3_hpc2', ':@computed_region_gxvr_9jxz'], axis=1)

In [27]:
bdlg_permits_raw['date_entered'] = pd.to_datetime(bdlg_permits_raw['date_entered'])
bdlg_permits_raw['date_issued'] = pd.to_datetime(bdlg_permits_raw['date_issued'])

In [28]:
bdlg_permits = bdlg_permits_raw[bdlg_permits_raw['zip'].isin(['37206', '37216', '37115'])]
bdlg_permits

,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,state,...,subdivision_lot,contact,per_ty,permit_subtype,ivr_trk,purpose,council_dist,census_tract,case_status,mapped_location
1,2020008381,Building Use & Occupancy,"Multifamily, Townhome",05100018100,2020-02-06,2021-11-24,1,609 CREATIVE WAY,MADISON,TN,...,SS CHERON ROAD WEST OF BRIARVILLE RD,"THUNDER RIVER CONSTRUCTION, LLC",CAUO,CAA03R301,3757751,Master permit for C & I village condos with 12...,5,37010802,EXPIRED,"{'latitude': '36.241870009', 'longitude': '-86..."
30,2020076899,Building Commercial - Rehab,"General Office, Professional Services",061110B00100CO,2020-12-09,2021-02-23,136413,4117 GALLATIN PIKE 101,NASHVILLE,TN,...,UNIT SUITE 1 INGLEWOOD PROFESSIONAL PARK,TRC CONSTRUCTION SERVICES INC,CACR,CAD02B016,3904314,suite 101 tenant improvement 1725 sq.ft. for T...,7,37011100,DONE,"{'latitude': '36.223268009', 'longitude': '-86..."
34,2021000520,Building Commercial - Tenant Finish Out,"Multifamily, Condominium 3&4 Unit Bldg",072150W03000CO,2021-01-05,2021-03-17,37967,1509 PORTER RD 201,NASHVILLE,TN,...,UNIT 201 PORTER VILLAGE PHASE II AMENDED,"TUDOR BUILDING GROUP, GP",CACT,CAA03R298,3911690,To finish out unit #201 1509 Porter Road . 924...,7,37011400,EXPIRED,"{'latitude': '36.194780979', 'longitude': '-86..."
50,2021005245,Building Sign Permit,Sign - Ground / Wall Signs,08309013400,2021-01-26,2021-03-18,4250,1000 MAIN ST,NASHVILLE,TN,...,LOTS 14 & 16 P/O LOT 15 BRANSFORD AND MCWHIRTE...,AAA SIGN ME UP LLC,CASN,CAZ02A001,3919103,to erect 15SF illuminated projecting sign and ...,6,37019200,EXPIRED,"{'latitude': '36.177996009', 'longitude': '-86..."
54,2021006138,Building Sign Permit,Sign - Ground / Wall Signs,07213038500,2021-01-29,2021-02-26,3560,2500 GALLATIN AVE,NASHVILLE,TN,...,LOT 1 CHESTER AVE SUB,TRI STAR TRANSPORT LLC,CASN,CAZ02A001,3920404,to erect 90SF face replacement on existing pyl...,6,37011700,EXPIRED,"{'latitude': '36.195368006', 'longitude': '-86..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33609,2023029004,Building Residential - Addition,Single Family Residence,07203034200,2023-05-03,2023-07-05,34293,1124 SHELTON AVE,NASHVILLE,TN,...,PT LOT 53 INGLEWOOD PLACE,SELF CONTRACTOR RESIDENTIAL (SEE APPLICANT INF...,CARA,CAA01R301,4316988,To construct an uncovered deck and stairs (710...,7,37011200,ISSUED,"{'latitude': '36.209294999', 'longitude': '-86..."
33613,2024001322,Building Residential - Rehab,Single Family Residence,08301008000,2024-01-05,2024-02-07,302449,1040 W GREENWOOD AVE,NASHVILLE,TN,...,LOT 7 JONES & BRADFORD,CJ MANAGEMENT & CONSULTING LLC,CARR,CAA01R301,4437128,Full Gut and Renovation to existing house. No ...,5,37011700,ISSUED,"{'latitude': '36.190321011', 'longitude': '-86..."
33619,2023099751,Building Residential - New,Detached Accessory Dwelling Unit,08309001900,2023-12-15,2024-02-06,89354,1211 B ORDWAY PL,NASHVILLE,TN,...,LOT 34 F O BEASLEY SUB BLK C DOUGLAS PROP,BUILDING COMPANY NUMBER 7 INC,CARN,CAA11R301,4429331,To construct DADU with 737 SF of heated/cooled...,6,37012100,ISSUED,"{'latitude': '36.180584005', 'longitude': '-86..."
33621,2023052084,Building Tree Removal Permit,Tree Removal Permit,083090K90000CO,2023-08-02,2023-08-02,10,201 B N 11TH ST,NASHVILLE,TN,...,COMMON AREA FIVE POINTS TOWNHOMES HOMEOWNERS A...,GRAYSTONE PROPERTIES LLC,CATR,CATREE,4357997,NaN,6,37019200,EXPIRED,"{'latitude': '36.178642002', 'longitude': '-86..."


In [29]:
bdlg_permits.to_csv('../data/clean/bldg_permits.csv')

... and MNPD calls<br><br>
Source: https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Calls-for-Servic/kwnd-qrrm/about_data

In [30]:
mnpd_export = client.get("kwnd-qrrm", limit=5000000)
mnpd_raw = pd.DataFrame.from_records(mnpd_export)
mnpd_raw.head()

ReadTimeout: HTTPSConnectionPool(host='data.nashville.gov', port=443): Read timed out. (read timeout=10)

...and Short Term Rental Permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Residential-Short-Term-Rental-Permits/2z82-v8pm/about_data

In [31]:
str_export = client.get("kwnd-qrrm", limit=100000)
strs = pd.DataFrame.from_records(str_export)
strs.head()

,event_number,call_rec,tencode,description,tencode_suf,tencode_suf_description,disposition_code,disposition_description,block,street_name,...,zone,rpa,lat,lon,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,complaint
0,PD202400139152,2024-02-19T23:59:31.000,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,A,ALARM,12,LOCATION/BUILDING SECURE,1300,BELL RD,...,331,8719,36.046,-86.675,"{'type': 'Point', 'coordinates': [-86.675, 36....",1,1,29,15,NaN
1,PD202400139151,2024-02-19T23:59:09.000,93,TRAFFIC VIOLATION,NaN,NaN,9,SUBJECT WARNED,700,STEWARTS FERRY PIKE,...,533,8955,36.153,-86.625,"{'type': 'Point', 'coordinates': [-86.625, 36....",1,1,7,38,NaN
2,PD202400139149,2024-02-19T23:58:55.000,53,HOLDUP / ROBBERY,P,PROGRESS,6,ASSISTED OTHER UNIT,4900,EDMONDSON PIKE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PD202400139148,2024-02-19T23:57:37.000,93,TRAFFIC VIOLATION,NaN,NaN,9,SUBJECT WARNED,NaN,E TRINITY LN,...,223,1861,36.206,-86.766,"{'type': 'Point', 'coordinates': [-86.766, 36....",1,1,13,16,NaN
4,PD202400139144,2024-02-19T23:55:25.000,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,PM,NaN,6,ASSISTED OTHER UNIT,3300,HAWKWOOD LN,...,721,2033,36.241,-86.769,"{'type': 'Point', 'coordinates': [-86.769, 36....",1,1,16,16,NaN
